In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

### Data

In [2]:
df = pd.read_csv('data_20180119_20180420.csv', index_col=0)
df.head()

,t0,T1,K1,C1,T2,K2,C2,S0,Adj_S0
0,2018-01-02,2018-01-19,1000.0,191.525,2018-04-20,800.0,396.725000,59.4505,1189.01001
1,2018-01-02,2018-01-19,1100.0,92.525,2018-04-20,1015.0,191.675000,59.4505,1189.01001
2,2018-01-02,2018-01-19,1150.0,46.075,2018-04-20,1025.0,183.025000,59.4505,1189.01001
3,2018-01-02,2018-01-19,1160.0,38.000,2018-04-20,1050.0,157.776341,59.4505,1189.01001
4,2018-01-02,2018-01-19,1170.0,30.450,2018-04-20,1080.0,137.850000,59.4505,1189.01001


In [3]:
S0 = df['Adj_S0'].unique()

In [4]:
t0List = list(df.t0.unique())

# dimension: K[k-th t0][t1 and t2][i-th option with i-th strike]
K = [[np.array(df.loc[df.t0 == t0,'K1']),np.array(df.loc[df.t0==t0,'K2'])] for t0 in t0List]

# dimension: Pi[k-th t0][t1 and t2][i-th option with i-th price]
Pi = [[np.array(df.loc[df.t0 == t0,'C1']),np.array(df.loc[df.t0==t0,'C2'])] for t0 in t0List]

In [5]:
# number of t0
N_t0 = len(t0List)

# number of times (t1, t2)
T = 2

# number of options in hedging
N1, N2 = 20, 20
Size = max(N1, N2)

In [6]:
# Assume marginal distribution is uniform
def margDistr(batch_size):
    data = np.random.uniform(size=(N_t0, batch_size, T))
    eps = 500
    for i in range(N_t0):
        for t in range(T):
            data[i,:,t] = (S0[i]-eps) + 2*eps*data[i,:,t]
    return data

When try with eps = 100, the prices will become negative. When try with eps = 500, the prices will become positive. Guess there is something to do with the range of simulated stock prices.

In [7]:
BATCH_SIZE = 1000
S = margDistr(BATCH_SIZE)

# different S for different t0: 13*Batch_size*2

In [8]:
Phi = (S[:,:,1]-S[:,:,0])
# define the payoff function of the derivative

### Variables

In [9]:
# parameter: d
d = tf.Variable(
    initial_value = tf.random.normal(shape=[N_t0, 1], dtype=tf.float64),
    trainable = True,
    name = 'd'
)

In [10]:
# parameter: lambda
Lambda = tf.Variable(
    initial_value = tf.random.normal(shape=[N_t0, T, Size], dtype=tf.float64),
    trainable = True,
    name = 'lambda'
)

In [11]:
# parameter: Delta_0
Delta_0 = tf.Variable(
    initial_value = tf.random.normal(shape=[N_t0, 1], dtype=tf.float64),
    trainable = True,
    name = 'Delta0'
)

In [12]:
# neural network: Delta_1(S_1)
Delta_1 = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=(BATCH_SIZE,)),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(N_t0, activation='linear', dtype=tf.float64)
])

### The step-by-step computation of the objective function

$d+\sum_{i=1}^{N_1}\lambda_{i,1}\Pi_{i,1}+\sum_{i=1}^{N_2}\lambda_{i,2}\Pi_{i,2}$

In [ ]:
hedgePrice = []
for n in range(N_t0):
    p = d[n]
    for t in range(T):
        p += tf.reduce_sum(Lambda[n,t,:]*Pi[n][t][:])
    hedgePrice.append(p)

In [ ]:
# take a look
hedgePrice

$d+\sum_{i=1}^{N_1}\lambda_{i,1}(S_1-K_{i,1})^++\sum_{i=1}^{N_2}\lambda_{i,2}(S_2-K_{i,2})^++\Delta_{0}\cdot(S_1-S_0)+\Delta_{1}(S_1)\cdot(S_2-S_1)$

In [ ]:
# one more variable Delta_0 Delta_0(S1-S0)

In [ ]:
hedgeTerm = []
for n in range(N_t0):
    p = d[n]
    for t in range(T):
        p += tf.reduce_sum(Lambda[n,t,:]*tf.maximum(S[n,:,t].reshape(BATCH_SIZE,1)-K[n][t][:],0),axis=1)
    p += Delta_0[n,0]*(S[n,:,0]-S0[n])
    p += Delta_1(S[n,:,0].reshape((1,-1)))[0,n]*(S[n,:,1]-S[n,:,0])
    hedgeTerm.append(p)

In [ ]:
# take a look
hedgeTerm

$\inf_{h\in\mathcal{H}^m}\int hd\mu_0+\int \beta_\gamma(c-h)d\theta$

$d+\sum_{i=1}^{N_1}\lambda_{i,1}\Pi_{i,1}+\sum_{i=1}^{N_2}\lambda_{i,2}\Pi_{i,2} + \Gamma\cdot\left[\left(\Phi(S_1,S_2)-d-\sum_{i=1}^{N_1}\lambda_{i,1}(S_1-K_{i,1})^+-\sum_{i=1}^{N_2}\lambda_{i,2}(S_2-K_{i,2})^+-\Delta_{0}\cdot(S_1-S_0)-\Delta_{1}(S_1)\cdot(S_2-S_1)\right)^+\right]^2$

In [ ]:
# objective function
Price = tf.reduce_mean(hedgePrice, axis=1)

Gamma = 100
diff = Phi - tf.stack(hedgeTerm, axis=0)
Penalty = tf.reduce_mean(tf.square(tf.nn.relu(diff)), axis=1)

ObjFunc = Price + Gamma*Penalty

In [ ]:
# take a look
Price

In [ ]:
# take a look
Penalty

In [ ]:
# take a look
ObjFunc

### Optimisation

In [13]:
# Combine all the computation as a function
def ObjFunc(d, Lambda, Delta_0, Delta_1, K, Pi, Phi, S):
    hedgePrice = []
    for n in range(N_t0):
        p = d[n]
        for t in range(T):
            p += tf.reduce_sum(Lambda[n,t,:]*Pi[n][t][:])
        hedgePrice.append(p)
    
    hedgeTerm = []
    for n in range(N_t0):
        p = d[n]
        for t in range(T):
            p += tf.reduce_sum(Lambda[n,t,:]*tf.maximum(S[n,:,t].reshape(BATCH_SIZE,1)-K[n][t][:],0),axis=1)
        p += Delta_0[n,0]*(S[n,:,0]-S0[n])
        p += Delta_1(S[n,:,0].reshape((1,-1)))[0,n]*(S[n,:,1]-S[n,:,0])
        hedgeTerm.append(p)
    
    Price = tf.reduce_mean(hedgePrice, axis=1)
    Gamma = 100
    diff = Phi-tf.stack(hedgeTerm, axis=0)
    Penalty = tf.reduce_mean(tf.square(tf.nn.relu(diff)), axis=1)
    
    return Price + Gamma*Penalty, Price

In [14]:
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = 0.01)

In [15]:
nIter = 10000
threshold = 1e-3
prev_price = tf.ones((N_t0,), dtype = tf.float64) * float('inf')

for step in range(nIter):
    if (step+1) % 500 == 0:
        print("Processing progress: %d / %d" %(step+1, nIter))
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(d)
        tape.watch(Lambda)
        tape.watch(Delta_0)
        for variable in Delta_1.trainable_variables:
            tape.watch(variable)

        obj, price = ObjFunc(d, Lambda, Delta_0, Delta_1, K, Pi, Phi, S)
    gradients_d, gradients_Lambda, gradients_Delta_0 = tape.gradient(obj, [d, Lambda, Delta_0])
    gradients_Delta_1 = tape.gradient(obj, Delta_1.trainable_variables)
    
    optimizer.apply_gradients(
        zip([gradients_d, gradients_Lambda, gradients_Delta_0] + gradients_Delta_1, 
            [d, Lambda, Delta_0] + Delta_1.trainable_variables)
    )
    
    diff = tf.abs(price - prev_price)
    if tf.reduce_max(diff) < threshold:
        print("Converged at step", step+1)
        break
    
    prev_price = price
    
    del tape

Processing progress: 500 / 10000
Processing progress: 1000 / 10000
Processing progress: 1500 / 10000
Processing progress: 2000 / 10000
Processing progress: 2500 / 10000
Processing progress: 3000 / 10000
Processing progress: 3500 / 10000
Processing progress: 4000 / 10000
Processing progress: 4500 / 10000
Processing progress: 5000 / 10000
Processing progress: 5500 / 10000
Processing progress: 6000 / 10000
Processing progress: 6500 / 10000
Processing progress: 7000 / 10000
Processing progress: 7500 / 10000
Processing progress: 8000 / 10000
Processing progress: 8500 / 10000
Processing progress: 9000 / 10000
Processing progress: 9500 / 10000
Processing progress: 10000 / 10000


In [16]:
price

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([ 572.15627692,  110.57773773,  665.79811309,  863.22225882,
        594.53214143,  581.65486932,  527.45265411,  846.40116891,
        188.1938056 ,  199.55496788, 1247.69049167,  677.00667534,
        306.72489957])>

In [17]:
obj

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([ 6665.24982372,   238.80760873,  3735.1870098 ,   864.6584886 ,
        4322.66391103,   581.65486932,   577.50697465, 29595.19114517,
        2076.89475379,   274.19511828,  1247.69049167,  8856.91732422,
        7492.96255468])>